In [1]:
import timesfm as tfm
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

2024-06-26 22:22:17.959540: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64:/opt/thinlinc/lib64:/opt/thinlinc/lib


In [2]:
tfm = tfm.TimesFm(
    context_len=320, # multiple of 32 (input_path_len) up to 512
    horizon_len=14, # anything but < context_len recommended
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend="cpu", # cpu or gpu or tpu
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

2024-06-26 22:22:41.005910: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Constructing model weights.


Constructed model weights in 2.49 seconds.
Restoring checkpoint from /home/x_yiyan/.cache/huggingface/hub/models--google--timesfm-1.0-200m/snapshots/8775f7531211ac864b739fe776b0b255c277e2be/checkpoints.


ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


Restored checkpoint in 1.32 seconds.
Jitting decoding.
Jitted decoding in 14.74 seconds.


In [9]:
data = pd.read_csv('traffic/traffic.csv')
data.head()

,date,0,1,2,3,4,5,6,7,8,...,852,853,854,855,856,857,858,859,860,OT
0,2016-07-01 02:00:00,0.0048,0.0146,0.0289,0.0142,0.0064,0.0232,0.0162,0.0242,0.0341,...,0.0051,0.0051,0.0074,0.0079,0.0051,0.0051,0.0339,0.0051,0.0100,0.0121
1,2016-07-01 03:00:00,0.0072,0.0148,0.0350,0.0174,0.0084,0.0240,0.0201,0.0338,0.0434,...,0.0036,0.0036,0.0107,0.0058,0.0036,0.0036,0.0348,0.0036,0.0087,0.0136
2,2016-07-01 04:00:00,0.0040,0.0101,0.0267,0.0124,0.0049,0.0170,0.0127,0.0255,0.0332,...,0.0030,0.0030,0.0043,0.0050,0.0030,0.0030,0.0327,0.0030,0.0061,0.0107
3,2016-07-01 05:00:00,0.0039,0.0060,0.0218,0.0090,0.0029,0.0118,0.0088,0.0163,0.0211,...,0.0033,0.0033,0.0019,0.0052,0.0033,0.0033,0.0292,0.0033,0.0040,0.0071
4,2016-07-01 06:00:00,0.0042,0.0055,0.0191,0.0082,0.0024,0.0095,0.0064,0.0087,0.0144,...,0.0049,0.0049,0.0011,0.0071,0.0049,0.0049,0.0264,0.0049,0.0040,0.0039


In [10]:
# DataFrame must have unique_id, ds and y columns
# Conversion of date to datetime format
data['ds'] = pd.to_datetime(data['date'])
data['unique_id'] = 0
data.rename(columns={'OT': 'y'},inplace=True)

In [11]:
data.head()

,date,0,1,2,3,4,5,6,7,8,...,854,855,856,857,858,859,860,y,ds,unique_id
0,2016-07-01 02:00:00,0.0048,0.0146,0.0289,0.0142,0.0064,0.0232,0.0162,0.0242,0.0341,...,0.0074,0.0079,0.0051,0.0051,0.0339,0.0051,0.0100,0.0121,2016-07-01 02:00:00,0
1,2016-07-01 03:00:00,0.0072,0.0148,0.0350,0.0174,0.0084,0.0240,0.0201,0.0338,0.0434,...,0.0107,0.0058,0.0036,0.0036,0.0348,0.0036,0.0087,0.0136,2016-07-01 03:00:00,0
2,2016-07-01 04:00:00,0.0040,0.0101,0.0267,0.0124,0.0049,0.0170,0.0127,0.0255,0.0332,...,0.0043,0.0050,0.0030,0.0030,0.0327,0.0030,0.0061,0.0107,2016-07-01 04:00:00,0
3,2016-07-01 05:00:00,0.0039,0.0060,0.0218,0.0090,0.0029,0.0118,0.0088,0.0163,0.0211,...,0.0019,0.0052,0.0033,0.0033,0.0292,0.0033,0.0040,0.0071,2016-07-01 05:00:00,0
4,2016-07-01 06:00:00,0.0042,0.0055,0.0191,0.0082,0.0024,0.0095,0.0064,0.0087,0.0144,...,0.0011,0.0071,0.0049,0.0049,0.0264,0.0049,0.0040,0.0039,2016-07-01 06:00:00,0


In [12]:
forecast_df = tfm.forecast_on_df(
    inputs=data,
    freq="H",  # monthly
    value_name="y",
    num_jobs=-1,
)
print(forecast_df)

Processing dataframe with multiple processes.


2024-06-26 22:42:16.953659: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64:/opt/thinlinc/lib64:/opt/thinlinc/lib


Finished preprocessing dataframe.
Finished forecasting.
    unique_id                  ds   timesfm  timesfm-q-0.1  timesfm-q-0.2  \
0           0 2018-07-02 02:00:00  0.013809       0.011174       0.012215   
1           0 2018-07-02 03:00:00  0.007853       0.005397       0.006520   
2           0 2018-07-02 04:00:00  0.004901       0.002033       0.003281   
3           0 2018-07-02 05:00:00  0.003901       0.001776       0.002440   
4           0 2018-07-02 06:00:00  0.003885       0.001394       0.002282   
5           0 2018-07-02 07:00:00  0.005035       0.002321       0.003447   
6           0 2018-07-02 08:00:00  0.007434       0.004928       0.006009   
7           0 2018-07-02 09:00:00  0.013432       0.010029       0.011147   
8           0 2018-07-02 10:00:00  0.019860       0.016706       0.018034   
9           0 2018-07-02 11:00:00  0.027094       0.023725       0.025007   
10          0 2018-07-02 12:00:00  0.036114       0.032003       0.033342   
11          0 2018-0